In [11]:
import json
import webbrowser
import threading
import pandas as pd
from dash import Dash, dcc, html, Input, Output

def get_data(year, category, order, month, json_file="dashboard_data_generation.json"):
    try:
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"The file '{json_file}' was not found.")
    except json.JSONDecodeError:
        raise ValueError(f"The file '{json_file}' is not in a valid JSON format.")
    key = f"Top 10 {category} {order}"
    
    try:
        return pd.DataFrame(data[year][month][key])
    except KeyError:
        raise KeyError(f"No data found for year '{year}', category '{category}', and order '{order}'.")

# Initialize Dash app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.Div([
        html.H1("Dashboard Zero Paper", style={
            'text-align': 'center',
            'background-color': '#f8f9fa',
            'padding': '20px',
            'margin-bottom': '10px',
            'border-bottom': '2px solid #ddd'
        })
    ], style={'width': '100%'}),
    html.Div([
        html.Div([
            html.Label('Año:'),
            dcc.Dropdown(
                id='dropdown-year',
                options=[{'label': '2024', 'value': '2024'}],
                value='2024',
                placeholder='Select a year',
                clearable=False
            )
        ], style={'padding': '10px', 'flex': '1'}),

        html.Div([
            html.Label('Institución:'),
            dcc.Dropdown(
                id='dropdown-category',
                options=[
                    {'label': 'Municipalidades', 'value': 'Municipalidades'},
                    {'label': 'Otros', 'value': 'Otros'}
                ],
                value='Municipalidades',
                placeholder='Select a category',
                clearable=False
            )
        ], style={'padding': '10px', 'flex': '1'}),

        html.Div([
            html.Label('Ranking:'),
            dcc.Dropdown(
                id='dropdown-order',
                options=[
                    {'label': 'Ascendente', 'value': 'Ascendente'},
                    {'label': 'Descendente', 'value': 'Descendente'}
                ],
                value='Ascendente',
                placeholder='Select the order',
                clearable=False
            )
        ], style={'padding': '10px', 'flex': '1'}),

        html.Div([
            html.Label('Mes:'),
            dcc.Dropdown(
                id='dropdown-month',
                options=[
                    {'label': 'Enero', 'value': 'Enero'},
                    {'label': 'Febrero', 'value': 'Febrero'},
                    {'label': 'Marzo', 'value': 'Marzo'},
                    {'label': 'Abril', 'value': 'Abril'},
                    {'label': 'Mayo', 'value': 'Mayo'},
                    {'label': 'Junio', 'value': 'Junio'},
                    {'label': 'Julio', 'value': 'Julio'},
                    {'label': 'Agosto', 'value': 'Agosto'},
                    {'label': 'Septiembre', 'value': 'Septiembre'},
                    {'label': 'Octubre', 'value': 'Octubre'}
                    #{'label': 'Noviembre', 'value': 'Noviembre'},
                    #{'label': 'Diciembre', 'value': 'Diciembre'}
                ],                
                value='Enero',
                placeholder='Select a month',
                clearable=False
            )
        ], style={'padding': '10px', 'flex': '1'})
    ], style={'display': 'flex', 'justify-content': 'space-between', 'flex-wrap': 'wrap', 'margin-bottom': '20px'}),

    html.Br(),

    html.Div([
        html.Div(id='expenses-table', style={'flex': '1', 'padding': '10px', 'overflow-x': 'auto', 'height': '500px', 'margin-right': '10px'}),
        html.Div(id='expenses-map', style={'flex': '1', 'height': '500px', 'margin-right': '10px'})
    ], style={'display': 'flex', 'justify-content': 'space-between', 'flex-wrap': 'wrap', 'height': '500px', 'width': '100%'})
])

# Callback to update table data
@app.callback(
    Output('expenses-table', 'children'),
    [Input('dropdown-year', 'value'),
     Input('dropdown-category', 'value'),
     Input('dropdown-order', 'value'),
     Input('dropdown-month', 'value')]
)

def update_table(year, category, order, month):
    try:
        df = get_data(year, category, order, month)
        df["Gasto"] = pd.to_numeric(df["Gasto"], errors='coerce')
        df = df.dropna(subset=["Gasto"])
        df["Gasto"] = df["Gasto"].apply(lambda x: f"{x:,.0f}" if isinstance(x, (int, float)) else x)
        df = df.drop(columns=['Latitud', 'Longitud'], errors='ignore')

        table = html.Table([
            html.Tr([html.Th(col, style={'border': '1px solid #ddd', 'padding': '8px', 'background-color': '#f2f2f2', 'text-align': 'center'}) for col in df.columns])
        ] + [
            html.Tr([html.Td(str(df.iloc[i][col]), style={'border': '1px solid #ddd', 'padding': '8px', 'text-align': 'center'}) for col in df.columns]) for i in range(len(df))
        ], style={'border': '1px solid #ddd', 'border-collapse': 'collapse', 'width': '100%'})

        return table

    except Exception as e:
        return html.Div(f"Error: {e}")

@app.callback(
    Output('expenses-map', 'children'),
    [Input('dropdown-year', 'value'),
     Input('dropdown-category', 'value'),
     Input('dropdown-order', 'value'),
     Input('dropdown-month', 'value')]
)
def display_map(year, category, order, month):
    try:
        df = get_data(year, category, order, month)
        # Filter the coordinates to be within Chile
        # Chile has latitudes between -17 and -56, and longitudes between -66 and -75
        df = df[(df['Latitud'] >= -56) & (df['Latitud'] <= -17) & 
                (df['Longitud'] >= -75) & (df['Longitud'] <= -66)]

        if 'Latitud' in df.columns and 'Longitud' in df.columns:
            df = df.dropna(subset=['Latitud', 'Longitud'])
            if df.empty:
                return html.Div("No data available for the map.")
                
            # Calculate the maximum and minimum values of latitude and longitude to adjust the zoom
            lat_min, lat_max = df["Latitud"].min(), df["Latitud"].max()
            lon_min, lon_max = df["Longitud"].min(), df["Longitud"].max()

            # Calculate the center of the map by taking the average of the coordinates
            lat_center = (lat_min + lat_max) / 2
            lon_center = (lon_min + lon_max) / 2

            # Calculate the zoom based on the size of the difference in latitudes and longitudes
            lat_diff = lat_max - lat_min
            lon_diff = lon_max - lon_min

            # Adjust the zoom so it doesn't zoom out too much
            if lat_diff > 10 or lon_diff > 10:
                zoom = 4  
            elif lat_diff > 5 or lon_diff > 5:
                zoom = 5 
            else:
                zoom = 6 

            fig = {
                "data": [
                    {
                        "lat": df["Latitud"],
                        "lon": df["Longitud"],
                        "type": "scattermapbox", 
                        "mode": "markers",  
                        "marker": {
                            "size": 12,
                            "color": "rgba(255, 0, 0, 0.8)", 
                            "opacity": 0.8,
                        },
                        "hovertemplate": (
                            "<b>Institución:</b> %{customdata[0]}<br>"
                            "<b>Gasto:</b> %{customdata[1]}<extra></extra>"
                        ),
                        "customdata": df[["Institución Padre", "Gasto"]].values,  
                    }
                ],
                "layout": {
                    "autosize": True,
                    "mapbox": {
                        "style": "carto-positron",  
                        "center": {"lat": lat_center, "lon": lon_center},  
                        "zoom": zoom, 
                        "scrollZoom": True,  
                    },
                    "height": 600,  
                    "margin": {"r": 0, "t": 0, "l": 0, "b": 0},  
                }
            }

            return dcc.Graph(
                id='mapa',
                figure=fig,
                config={'scrollZoom': True, 'displayModeBar': False},
            )
        else:
            return html.Div("The columns 'Latitude' and 'Longitude' are not available in the data.")
    except Exception as e:
        return html.Div(f"Error loading the map: {e}")
        

def run_app(app, port):
    try:
        app.run_server(debug=True, port=port)
    except OSError:  # If the port is occupied, try a different port
        print(f"Port {port} is occupied, trying another one...")
        app.run_server(debug=True, port=port + 1)
        
def open_browser(port):
    webbrowser.open(f"http://127.0.0.1:{port}", new=2)

if __name__ == '__main__':
    default_port = 8052
    threading.Timer(1, open_browser, [default_port]).start()
    run_app(app, default_port)